##### Import Section

In [25]:
import os
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import pandas as pd
from PIL import Image
from torchvision.utils import save_image
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
from pathlib import Path
from  torchinfo import summary

##### Config Section

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 3e-4
batch_size = 32
num_epochs = 10

##### Data Processing Section

In [27]:
class CustomDataset(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [28]:
def data_loader(csv_path, image_dir, transform = None, batch_size=64):
    dataset =   CustomDataset(
        csv_path=csv_path,
        root_dir=image_dir,
        transform=transform,
    )
    train_set, test_set = torch.utils.data.random_split(dataset, [5, 5])
    train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

    return (dataset, train_loader, test_loader)

In [29]:
# Path
current_working_dir = Path.cwd()
csv_path = os.path.join(current_working_dir, "data", "cats_dogs.csv")
image_dir = os.path.join(current_working_dir, "data", "cats_dogs_resized")
# csv_path, cats_dogs_csv, image_path

In [50]:
to_transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),  # Resizes (32,32) to (36,36)
        transforms.RandomCrop((224, 224)),  # Takes a random (32,32) crop
        transforms.ColorJitter(brightness=0.5),  # Change brightness of image
        transforms.RandomRotation(
            degrees=45
        ),  # Perhaps a random rotation from -45 to 45 degrees
        transforms.RandomHorizontalFlip(
            p=0.5
        ),  # Flips the image horizontally with probability 0.5
        transforms.RandomVerticalFlip(
            p=0.05
        ),  # Flips image vertically with probability 0.05
        transforms.RandomGrayscale(p=0.2),  # Converts to grayscale with probability 0.2
        transforms.ToTensor(),  # Finally converts PIL image to tensor so we can train w. pytorch
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)
dataset, train_loader, test_loader = data_loader(
    csv_path, image_dir, transform=to_transform
)

In [51]:
img_num=0
for _ in range(10):
    for img, label in dataset:
        if label == 0:
            name = "cat"
        else:
            name = "dog"
        save_image(img, name + str(img_num)+'.png')
        img_num += 1

##### Model Creation Section

In [ ]:
# create CNN layer:
class CNNNet(nn.Module):
    def __init__(self, in_channels = 3, num_classes=10):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels= 6, kernel_size=(5,5)) # output = (32-5+2*0)/1+1=28 -> [1, 6,28,28] parameter: 5^2*3*6+6 = 456 and colROw: 456*28*28
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) # output= 28-2/2+1=13+1= 14 => [1,6,14,14] params =null col_Row=null
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels= 16, kernel_size=(5,5)) # output = 14-5/1 + 1=10=>[1,16,10,10] parameter = 5^2*6*16+16 = 2416 and colRow = 2416*10*10
        self.conv3 = nn.Conv2d(16,120,5)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(120,64)
        self.fc2 = nn.Linear(64,num_classes)
    
    def forward(self,x):
        x = F.relu(self.conv1(x)) # output = (32-5+2*0)/1+1=28 -> [1, 6,28,28] parameter: 5^2*3*6+6 = 456 and colROw: 456*28*28
        x = self.pool(x)  # output= 28-2/2+1=13+1= 14 => [1,6,14,14] params =null col_Row=null
        x = F.relu(self.conv2(x)) # output = 14-5/1 + 1=10=>[1,16,10,10] parameter = 5^2*6*16+16 = 2416 and colRow = 2416*10*10
        x = self.pool(x) # output= 10-2/2+1= 4+1= 5 => [1,16,5,5] params =null col_Row=null
        x = F.relu(self.conv3(x)) # output = 5-5/1 + 1= 1 =>[1,120,1,1]; parameter = 5^2*16*120+120 = 48120 and colRow = 48120*1*1
        # x = x.reshape(x.shape[0], -1)
        x = self.flat(x) # output= [1, 120*1*1]=>[1, 120]; params = null;  colRow =null
        x = F.relu(self.fc1(x)) # output:[1, 64]; params: 120*64+64= 7744; colRow= 7744*1=7744
        x = self.fc2(x) # output:[1, 10]; params: 64*10+10= 650; colRow =650
        return x

##### Train and Evaluation section

In [ ]:
def tain_nn(train_loader, model, criterion, optimizer):
    for epoch in range(num_epochs):
        losses = []

        for batch_idx, (data, targets) in enumerate(train_loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            losses.append(loss.item())

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()

        print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

##### Initialize model and trainning

In [ ]:
model = CNNNet().to(device)
summary(model, input_size=(1, 3, 32, 32), col_names= ("input_size", "output_size", "num_params", "mult_adds"))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
CNNNet                                   [1, 3, 32, 32]            [1, 10]                   --                        --
├─Conv2d: 1-1                            [1, 3, 32, 32]            [1, 6, 28, 28]            456                       357,504
├─MaxPool2d: 1-2                         [1, 6, 28, 28]            [1, 6, 14, 14]            --                        --
├─Conv2d: 1-3                            [1, 6, 14, 14]            [1, 16, 10, 10]           2,416                     241,600
├─MaxPool2d: 1-4                         [1, 16, 10, 10]           [1, 16, 5, 5]             --                        --
├─Conv2d: 1-5                            [1, 16, 5, 5]             [1, 120, 1, 1]            48,120                    48,120
├─Flatten: 1-6                           [1, 120, 1, 1]            [1, 120]                  --                        --
├─L

In [ ]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)